In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
#import quandl

import cvxportfolio as cp

获取股票价格数据及无风险利率

In [2]:
from zipline.research import returns
from zipline.data.treasuries_cn import get_treasury_data

In [3]:
# 注意无风险利率选择一年期，然后除每年交易天数244

tickers = ['300001', '000333', '600771', '600645']
start ='2016-01-01'
end ='2018-05-25'
rets = returns(tickers, start, end)

[2018-05-28 04:11:23.561556] INFO: zipline.finance.metrics.tracker: 模拟1个交易日
首个开盘时间: 2018-05-25 01:31:00+00:00
最后收盘时间: 2018-05-25 07:00:00+00:00


In [4]:
rets.head()

,特锐德(300001),美的集团(000333),广誉远(600771),中源协和(600645)
2016-01-04 00:00:00+00:00,-0.099644,-0.094482,0.0,-0.099985
2016-01-05 00:00:00+00:00,-0.055950,0.054211,0.0,-0.032473
2016-01-06 00:00:00+00:00,0.020717,0.008265,0.0,0.047871
2016-01-07 00:00:00+00:00,-0.100167,-0.080731,0.0,-0.099966
2016-01-08 00:00:00+00:00,-0.021627,0.026527,0.0,0.065368


We compute rolling estimates of the first and second moments of the returns using a window of 250 days. We shift them by one unit (so at every day we present the optimizer with only past data).

计算滚动244天

In [5]:
r_hat = rets.rolling(window=244, min_periods=244).mean().shift(1).dropna()
Sigma_hat = rets.rolling(window=244, min_periods=244, closed='neither').cov().dropna()

r_hat.tail()

,特锐德(300001),美的集团(000333),广誉远(600771),中源协和(600645)
2018-05-21 00:00:00+00:00,-0.001113,0.002156,0.001664,0.000444
2018-05-22 00:00:00+00:00,-0.000842,0.002035,0.001665,0.000487
2018-05-23 00:00:00+00:00,-0.000693,0.001857,0.001810,0.000819
2018-05-24 00:00:00+00:00,-0.000614,0.001780,0.001805,0.000991
2018-05-25 00:00:00+00:00,-0.000689,0.001733,0.001766,0.000936


In [6]:
Sigma_hat.tail()

特锐德(300001)  美的集团(000333)  \
2018-05-24 00:00:00+00:00 中源协和(600645)     0.000131     -0.000017   
2018-05-25 00:00:00+00:00 特锐德(300001)      0.000382      0.000050   
                          美的集团(000333)     0.000050      0.000464   
                          广誉远(600771)      0.000063      0.000053   
                          中源协和(600645)     0.000131     -0.000014   

                                        广誉远(600771)  中源协和(600645)  
2018-05-24 00:00:00+00:00 中源协和(600645)     0.000036      0.000713  
2018-05-25 00:00:00+00:00 特锐德(300001)      0.000063      0.000131  
                          美的集团(000333)     0.000053     -0.000014  
                          广誉远(600771)      0.000314      0.000032  
                          中源协和(600645)     0.000032      0.000715

Here we define the transaction cost and holding cost model (sections 2.3 and 2.4 [of the paper](https://web.stanford.edu/~boyd/papers/cvx_portfolio.html)). The data can be expressed 
as 
- a scalar (like we're doing here), the same value for all assets and all time periods;
- a Pandas Series indexed by the asset names, for asset-specific values; 
- a Pandas DataFrame indexed by timestamps with asset names as columns, for values that vary by asset and in time.

定义交易成本及持有成本
- 标量：所有时间及资产一个固定值
- 序列：特定资产固定值
- 数据框：随时间及资产而变化的值

In [7]:
tcost_model=cp.TcostModel(0)
hcost_model=cp.HcostModel(0)

We define the single period optimization policy (section 4 [of the paper](https://web.stanford.edu/~boyd/papers/cvx_portfolio.html)). 

In [8]:
risk_model = cp.FullSigma(Sigma_hat)
gamma_risk, gamma_trade, gamma_hold = 1., 1., 1.
leverage_limit = cp.LeverageLimit(1)

spo_policy = cp.SinglePeriodOpt(return_forecast=r_hat, 
                                costs=[],
                                constraints=[leverage_limit])

In [9]:
t = r_hat.index[2]
p_0 = pd.Series(index=rets.columns, data=100000) # 期初投资组合
z = spo_policy.get_trades(p_0, t)
z

特锐德(300001)     -99999.999669
美的集团(000333)    299999.997111
广誉远(600771)     -99999.997490
中源协和(600645)    -99999.999952
dtype: float64